## 타이타닉 데이터 다운로드

In [3]:
!pip install kaggle
from google.colab import files
files.upload()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"taeyangg4","key":"b9a22b3cb91a965bc5ca1c46065ee041"}'}

In [5]:
# ls -1ha kaggle.json

In [6]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600  ~/.kaggle/kaggle.json

In [7]:
!kaggle competitions download -c titanic

  0% 0.00/34.1k [00:00<?, ?B/s]
100% 34.1k/34.1k [00:00<00:00, 19.7MB/s]


In [10]:
# !ls
!unzip /content/titanic.zip

Archive:  /content/titanic.zip
  inflating: gender_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


# import & sklearn 

In [21]:
# !pip list
!pip install xgboost
!pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 KB 4.5 MB/s eta 0:00:00


In [293]:
# 데이터 관리
import pandas as pd
import numpy as np
import pickle
import datetime
import pytz
import time
import math
import os

# 시각화
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm

# 모델링
from xgboost import XGBClassifier
from category_encoders import OneHotEncoder, OrdinalEncoder, TargetEncoder
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [202]:
data = pd.read_csv('/content/train.csv')

In [203]:
test_data = pd.read_csv('/content/test.csv')

# EDA 및 전처리

In [204]:
df = data.copy()

In [206]:
test = test_data.copy()

# 모델링 작업

In [194]:
class My_func:

  # __init__
  def __init__(self):
    self.result = 0
    self._features = []
    self._target = ''

  # 특성 및 타겟 setter, getter 작성
  @property
  def features(self):
    return self._features
  
  @features.setter
  def features(self, data):
    values = data.drop(columns = self._target).columns
    self._features = list(values)

  @property
  def target(self):
    return self._target
  
  @target.setter
  def target(self, value):
    target = value
    self._target = target

  # 훈련/검증/테스트 데이터 나누는 함수
  def split_data(self, data):
    train, test = train_test_split(data, train_size=0.80, test_size=0.20, random_state=2)
    return train, test

  # 특성과 타겟값 분리하는 함수
  def split_target(self, data, features, target):
    X_data = data[features]
    y_data = data[target]

    return X_data, y_data

  # 인코딩하는 함수
  def encoding(self, X_train, X_test):
    ord = OrdinalEncoder()

    X_train_enc = ord.fit_transform(X_train)
    X_test_enc = ord.transform(X_test)

    return X_train_enc, X_test_enc

  # 생존자예측 XGBClassifier 모델 함수
  def modeling(self):
    xgb_model = XGBClassifier(
        max_depth=3, 
        learning_rate=0.1, 
        n_estimators=100, 
        random_state=42
        )
    
    return xgb_model

In [195]:
# 클래스 선언
fuc = My_func()
fuc.target = 'Survived'
fuc.features = df

# fuc.target, fuc.features

In [196]:
# 함수 적용
train, val = fuc.split_data(df) # 훈련/검증 데이터로 나누기
X_train, y_train = fuc.split_target(train, fuc.features, fuc.target) # 훈련데이터 특성과 타겟값을 분리
X_val, y_val = fuc.split_target(val, fuc.features, fuc.target) # 검증데이터 특성과 타겟값을 분리
X_train_enc, X_val_enc = fuc.encoding(X_train, X_val) # 인코딩 작업

# X_train_enc.shape, y_train.shape, X_val_enc.shape, y_val.shape

In [197]:
# 모델링
model = fuc.modeling()
model.fit(X_train_enc , y_train)

XGBClassifier(random_state=42)

In [198]:
y_pred = model.predict(X_val_enc)

In [200]:
acc = accuracy_score(y_pred, y_val)
print(f'정확도 : {acc}')

정확도 : 0.7821229050279329


# 테스트셋으로 예측한후 내보내기

In [211]:
df = data.copy()
X_test = test_data.copy()

In [212]:
# 클래스 선언
fuc_test = My_func()
fuc_test.target = 'Survived'
fuc_test.features = df

# fuc.target, fuc.features

In [213]:
# 함수 적용
X_train, y_train = fuc_test.split_target(train, fuc_test.features, fuc_test.target) # 훈련데이터 특성과 타겟값을 분리
X_train_enc, X_test_enc = fuc_test.encoding(X_train, X_test) # 인코딩 작업

In [214]:
# 모델링
model = fuc.modeling()
model.fit(X_train_enc , y_train)

XGBClassifier(random_state=42)

In [216]:
y_pred_test = model.predict(X_test_enc)

# 파일뒤에 현재시간 기록하기

In [294]:
# Get the current time in the UTC+9 timezone
now_utc_9 = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))

# Format the datetime object as a string
utc_9_time_str = now_utc_9.strftime('%Y-%m-%d-%H-%M-%S')

# 모델 피클링해서 내보내기

In [297]:
import pickle

model_name = f'model_{utc_9_time_str}.pkl'
with open(model_name,'wb') as pickle_file:
    pickle.dump(model, pickle_file)

files.download(model_name )

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# submission파일 내컴퓨터로 내보내기

In [298]:
from google.colab import files

file_name = f"titanic_submission_{utc_9_time_str}.csv"
pd.DataFrame(y_pred_test).to_csv(file_name, index=False)
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>